In [157]:
import numpy as np  
import pandas as pd
import tqdm

In [144]:
#Create sample hiring data

def hiring_data(shift):
    
    n = 1000
    U = np.random.normal(0, 1, n)
    X = np.random.binomial(1,np.exp(U)/(1+np.exp(U)),n)
    Z = np.random.binomial(1,np.exp(U)/(1+np.exp(U)),n)
    W = np.random.binomial(1,0.3,n)
    Y = np.random.binomial(1,(2/5)*(X+Z-2*X*Z)+(1/6)*W+0.1,n)

    sample_1 = pd.DataFrame({'X':X, 'Z':Z, 'W':W, 'Y':Y})

    if shift == 1:
        U = np.random.normal(0, 1, n)
        X = np.random.binomial(1,np.exp(U)/(1+np.exp(U)),n)
        Z = np.random.binomial(1,np.exp(U)/(1+np.exp(U)),n)
        W = np.random.binomial(1,0.3,n)
        Y = np.random.binomial(1,(1/5)*(X+Z)+(2/6)*W,n)

        sample_2 = pd.DataFrame({'X':X, 'Z':Z, 'W':W, 'Y':Y})
    
    else:

        n = 1000
        U = np.random.normal(0, 1, n)
        X = np.random.binomial(1,np.exp(U)/(1+np.exp(U)),n)
        Z = np.random.binomial(1,np.exp(U)/(1+np.exp(U)),n)
        W = np.random.binomial(1,0.3,n)
        Y = np.random.binomial(1,(2/5)*(X+Z-2*X*Z)+(1/6)*W+0.1,n)

        sample_2 = pd.DataFrame({'X':X, 'Z':Z, 'W':W, 'Y':Y})
    
    return sample_1, sample_2

sample_1, sample_2 = hiring_data(0)
_ , sample_3 = hiring_data(1)

        


In [136]:
def shuffle_test(sample_1, sample_2, type, n_simulations):
    
    group_cols = ['X', 'Z', 'W']
    test_statistic = ((sample_1.groupby(group_cols).mean()['Y'] - sample_2.groupby(group_cols).mean()['Y']).reset_index()['Y'])
    results_table = pd.DataFrame({'X':np.zeros(len(test_statistic)), 'Z':np.zeros(len(test_statistic)), 'W':np.zeros(len(test_statistic)),
                                  'test_statistic':test_statistic, 'p_value':np.zeros(len(test_statistic)),'decision':np.zeros(len(test_statistic)),
                                  'Bonferroni-Holm p-value':np.zeros(len(test_statistic)), 'Bonferroni-Holm decision':np.zeros(len(test_statistic))})
    results_table[['X','Z','W']] = sample_1.groupby(group_cols).mean().reset_index()[['X', 'Z', 'W']]
    pooled_data = pd.concat([sample_1, sample_2])
    p_value = np.zeros(len(test_statistic))

    if type == 'bootstrap':
        #array of size len(test_statistic) x n_simulations
        bootstrap_statistic = np.zeros((len(test_statistic), n_simulations))
        for i in range(n_simulations):
            bootstrap_sample = pooled_data.sample(frac=1, replace=True)
            #first n=500 rows are sample_1, next m=500 rows are sample_2
            bootstrap_sample_1 = bootstrap_sample.iloc[:len(sample_1)]
            bootstrap_sample_2 = bootstrap_sample.iloc[len(sample_1):]
            bootstrap_statistic[:,i] = ((bootstrap_sample_1.groupby(group_cols).mean()['Y'] - bootstrap_sample_2.groupby(group_cols).mean()['Y']).reset_index()['Y'])
        
        for i in range(len(test_statistic)):
            p_value[i] = np.sum(np.abs(bootstrap_statistic[i]) > np.abs(test_statistic[i]))/n_simulations
            results_table.loc[i,'p_value'] = p_value[i]
            if p_value[i] < 0.05:
                results_table.loc[i,'decision'] = "rejected"
            else:
                results_table.loc[i,'decision'] = 'not rejected'

        sorted_p_values_indices = results_table['p_value'].sort_values().index
        for i in range(len(test_statistic)):
            index = sorted_p_values_indices[i]
            temp_p_value = results_table['p_value'][index]
            bonferroni_holm_factor = (len(test_statistic) - (i+1) + 1)
            bonferroni_holm_p_value = temp_p_value * bonferroni_holm_factor
            if bonferroni_holm_p_value > 1:
                bonferroni_holm_p_value = 1

            if bonferroni_holm_p_value < 0.05:
                results_table.loc[index,'Bonferroni-Holm decision'] = "rejected"
                results_table.loc[index,'Bonferroni-Holm p-value'] = bonferroni_holm_p_value
            else:
                results_table.loc[index,'Bonferroni-Holm decision'] = 'not rejected'
                results_table.loc[index,'Bonferroni-Holm p-value'] = bonferroni_holm_p_value
                break

        return results_table

            
    if type == 'permutation':
        #array of size len(test_statistic) x n_simulations
        permutation_statistic = np.zeros((len(test_statistic), n_simulations))
        for i in range(n_simulations):
            permutation_sample = pooled_data.sample(frac=1)
            #first n=500 rows are sample_1, next m=500 rows are sample_2
            permutation_sample_1 = permutation_sample.iloc[:len(sample_1)]
            permutation_sample_2 = permutation_sample.iloc[len(sample_1):]
            permutation_statistic[:,i] = ((permutation_sample_1.groupby(group_cols).mean()['Y'] - permutation_sample_2.groupby(group_cols).mean()['Y']).reset_index()['Y'])

        for i in range(len(test_statistic)):
            p_value[i] = np.sum(np.abs(permutation_statistic[i]) > np.abs(test_statistic[i]))/n_simulations
            results_table.loc[i,'p_value'] = p_value[i]
            if p_value[i] < 0.05:
                results_table.loc[i,'decision'] = "rejected"
            else:
                results_table.loc[i,'decision'] = 'not rejected'

        sorted_p_values_indices = results_table['p_value'].sort_values().index
        for i in range(len(test_statistic)):
            index = sorted_p_values_indices[i]
            temp_p_value = results_table['p_value'][index]
            bonferroni_holm_factor = (len(test_statistic) - (i+1) + 1)
            bonferroni_holm_p_value = temp_p_value * bonferroni_holm_factor
            if bonferroni_holm_p_value > 1:
                bonferroni_holm_p_value = 1
        
            if bonferroni_holm_p_value < 0.05:
                results_table.loc[index,'Bonferroni-Holm decision'] = "rejected"
                results_table.loc[index,'Bonferroni-Holm p-value'] = bonferroni_holm_p_value
            else:
                results_table.loc[index,'Bonferroni-Holm decision'] = 'not rejected'
                results_table.loc[index,'Bonferroni-Holm p-value'] = bonferroni_holm_p_value
                break



        return results_table

            


In [158]:
number_simulations = [100,10000,50000,100000,250000,500000,1000000]

#run test (sample_1 vs sample_2 and sample_1 vs sample_3) both bootstrap and permutation for all simulations in number_simulations and
#store results tables in seperate .csv files with naming convention 'sample_1_vs_sample2_n_simulations_bootstrap.csv', 'sample_1_vs_sample2_n_simulations_permutation.csv' and so on
#we use tqdm to track progress of the simulations
for n in tqdm.tqdm(number_simulations, desc='Running simulations'):
    results_table_bootstrap = shuffle_test(sample_1, sample_2, 'bootstrap', n)
    results_table_permutation = shuffle_test(sample_1, sample_2, 'permutation', n)
    results_table_bootstrap.to_csv('sample_1_vs_sample2_'+str(n)+'_bootstrap.csv')
    results_table_permutation.to_csv('sample_1_vs_sample2_'+str(n)+'_permutation.csv')
    
    results_table_bootstrap = shuffle_test(sample_1, sample_3, 'bootstrap', n)
    results_table_permutation = shuffle_test(sample_1, sample_3, 'permutation', n)
    results_table_bootstrap.to_csv('sample_1_vs_sample3_'+str(n)+'_bootstrap.csv')
    results_table_permutation.to_csv('sample_1_vs_sample3_'+str(n)+'_permutation.csv')






Running simulations:   0%|          | 0/7 [00:00<?, ?it/s]C:\Users\41782\AppData\Local\Temp\ipykernel_58916\4191344351.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'not rejected' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  results_table.loc[i,'decision'] = 'not rejected'
C:\Users\41782\AppData\Local\Temp\ipykernel_58916\4191344351.py:43: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'not rejected' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  results_table.loc[index,'Bonferroni-Holm decision'] = 'not rejected'
C:\Users\41782\AppData\Local\Temp\ipykernel_58916\4191344351.py:66: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'not rejected' has dtype

,X,Z,W,test_statistic,p_value,decision,Bonferroni-Holm p-value,Bonferroni-Holm decision
0,0,0,0,0.019557,0.500612,not rejected,0.000000,0.0
1,0,0,1,-0.025229,0.695558,not rejected,0.000000,0.0
2,0,1,0,-0.067901,0.246772,not rejected,0.000000,0.0
3,0,1,1,0.118885,0.154284,not rejected,0.000000,0.0
4,1,0,0,0.002795,0.957500,not rejected,0.000000,0.0
5,1,0,1,-0.040441,0.674940,not rejected,0.000000,0.0
6,1,1,0,-0.055369,0.048842,rejected,0.390736,not rejected
7,1,1,1,-0.067412,0.327638,not rejected,0.000000,0.0
